**Kotlin Notebook 플러그인 설치해야 보이는 점 잊지말자**

In [ ]:
// 이하 두 클래스를 수정하지 않고 login 를 호출하고 싶다.
data class User(val id: Int)
data class Admin(val id: Int)

// User.login()
// Admin.login()

- 각 클래스 외부에서 호출 가능해야한다.
- Mixin 클래스를 따로 정의할 경우, 그 클래스는 상태를 갖을 수 없다.

[이 블로그](https://benoitaverty.com/articles/en/kotlin-mixins-using-delegation) 가 참고가 되긴 하는데...
이 식대로 하자면 User 와 Admin 클래스 선언에 뭔가를 붙여야한다.

User, Admin 관리하는 사람은 따로 있고, 인증서버 다루는 사람도 따로 있으면 못 건드리니까
이런 문제를 해결할 수 있는 방법을 생각해보고 실험해보자


In [ ]:
// 컴포지션 방식
import kotlin.reflect.KClass
import kotlin.reflect.KProperty

// 기본 클래스 ------------------------------------------
data class User(val id: Int)
data class Admin(val id: Int)
data class Cert(val id: Int) // 이런거도 추가하자면

sealed class Authorization {
    class Empty() : Authorization()
    class Success(val role: String) : Authorization()
}
// ----------------------------------------------------

// 요기는 auth 라이브러리 --------------------------------
class Authorizor {
    fun <T> authorize(target: T, mixin: LoginMixin<T>): Authorization {
        // 의미론적으로 중복되는 것 같아
        val mapper = object : LoginMixin<T> by mixin {
            fun login() = target.login()
        }

        return mapper.login()
    }
}

interface LoginMixin<T> {
    fun T.login(): Authorization
}

// ----------------------------------------------------

// 이하 기본 클래스들과 auth 라이브러리를 땡겨쓰는 곳

// authorizor.authorize 를 호출하려면 Mixin 구현을 강제할 수 있긴하다.
// 어댑터 레이어에서 이런 믹스인 만든다고 하면 유의미한 구조인가
object UserLoginMixin : LoginMixin<User> {
    override fun User.login(): Authorization {
        return Authorization.Success("유저의 권한! 당신의 아이디는 ${this.id}")
    }
}

object AdminLoginMixin : LoginMixin<Admin> {
    override fun Admin.login(): Authorization {
        return Authorization.Success("어드민의 권한! 당신의 아이디는 ${this.id}")
    }
}

object CertLoginMixin : LoginMixin<Cert> {
    override fun Cert.login(): Authorization {
        return Authorization.Success("인증서의 권한! 인증서의 아이디는 ${this.id}")
    }
}

val authorizor = Authorizor()

val user = User(1)
println("User 인증 결과는! ${authorizor.authorize(user, UserLoginMixin)}")

val admin = Admin(2)
println("AdminLoginMixin 인증 결과는! ${authorizor.authorize(admin, AdminLoginMixin)}")

val cert = Cert(3)
println("CertLoginMixin 인증 결과는! ${authorizor.authorize(cert, CertLoginMixin)}")


흠... 새로운 아이디어가 있으면 추가해보자.